In [4]:
# loading packages
library(repr)
library(tidyverse)
library(tidymodels)
# library(themis)
# set.seed(31)

stroke <- read_csv("data/stroke-data.csv")
stroke_clean <- stroke %>%
    select(age, hypertension, heart_disease, avg_glucose_level, bmi, stroke) %>%
    mutate(stroke = as_factor(stroke),
           bmi = as.numeric(bmi)) 

Parsed with column specification:
cols(
  id = col_double(),
  gender = col_character(),
  age = col_double(),
  hypertension = col_double(),
  heart_disease = col_double(),
  ever_married = col_character(),
  work_type = col_character(),
  Residence_type = col_character(),
  avg_glucose_level = col_double(),
  bmi = col_character(),
  smoking_status = col_character(),
  stroke = col_double()
)

Warning message:
“Problem with `mutate()` input `bmi`.
ℹ NAs introduced by coercion
ℹ Input `bmi` is `as.numeric(bmi)`.”
Warning message in mask$eval_all_mutate(dots[[i]]):
“NAs introduced by coercion”


In [11]:
stroke_split <- initial_split(stroke_clean, prop = 0.75, strata = stroke)
stroke_train <- training(stroke_split)
stroke_test <- testing(stroke_split)

In [12]:
for (i in 1:10) {
    up_recipe <- recipe(stroke ~ ., data = stroke_train) %>%
        step_scale(all_predictors()) %>%
        step_center(all_predictors()) %>%
        step_upsample(stroke, over_ratio = i * 0.1, skip = FALSE) %>%
        prep()
    upsampled_stroke <- bake(up_recipe, stroke_train)
    
    stroke_recipe <- recipe(stroke ~ avg_glucose_level + age + hypertension, data = upsampled_stroke) %>%
      step_scale(all_predictors()) %>%
      step_center(all_predictors())
    
    tune_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
      set_engine("kknn") %>%
      set_mode("classification")
    
    knn_results <- workflow() %>%
      add_recipe(stroke_recipe) %>%
      add_model(knn_spec) %>%
      tune_grid(resamples = stroke_vfold, grid = k_vals) %>%
      collect_metrics() %>% # Plot accuracies vs neighbours
      filter(.metric == "accuracy") %>%
      filter(mean == max(mean)) %>%
      slice(1) %>%
      pull(neighbors)
    
    kmin <- 
    
    knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = kmin) %>%
      set_engine("kknn") %>%
      set_mode("classification")

    # workflow + fit data
    knn_fit <- workflow() %>%
      add_recipe(stroke_recipe) %>%
      add_model(knn_spec) %>%
      fit(data = upsampled_stroke)

    stroke_test_predictions1 <- predict(knn_fit, stroke_test) %>%
      bind_cols(stroke_test) %>%
      metrics(truth = stroke, estimate = .pred_class)
    print(i)
    print(stroke_test_predictions1)
}

[1] 1
# A tibble: 2 x 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
1 accuracy binary         0.955
2 kap      binary         0    
[1] 2
# A tibble: 2 x 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
1 accuracy binary       0.115  
2 kap      binary       0.00706
[1] 3
# A tibble: 2 x 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
1 accuracy binary        0.0446
2 kap      binary        0     
[1] 4
# A tibble: 2 x 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
1 accuracy binary        0.0446
2 kap      binary        0     
[1] 5
# A tibble: 2 x 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
1 accuracy binary        0.0446
2 kap      binary        0     
[1] 6
# A tibble: 2 x 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
1 accuracy binary        0.0446
2 kap      binary        0     
[1] 7
# A tibble: 2 x 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
